# Experimenting with Pytorch Hooks
---

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Build Model

In [2]:
class MNISTConvNet(nn.Module):

    def __init__(self):
        # this is the place where you instantiate all your modules
        # you can later access them using the same names you've given them in
        # here
        super(MNISTConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, input):
        # it's the forward function that defines the network structure
        # we're accepting only a single input in here, but if you want,
        # feel free to use more
        x = self.pool1(F.relu(self.conv1(input)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

In [33]:
model = MNISTConvNet()
print(model)

MNISTConvNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


## Model Analysis

In [34]:
inp = torch.randn(1,1,28,28)
out = model(inp)
out.shape

torch.Size([1, 10])

In [35]:
target = torch.tensor([3], dtype=torch.long)
loss_fn = nn.CrossEntropyLoss()  # LogSoftmax + ClassNLL Loss
err = loss_fn(out, target)
err.backward()
err

tensor(2.3587, grad_fn=<NllLossBackward>)

In [36]:
model.conv1.weight.grad.size(), model.conv1.weight.data.size()

(torch.Size([10, 1, 5, 5]), torch.Size([10, 1, 5, 5]))

## Hooks: Inspecting Output or Grad of a single layer

In [37]:
def printnorm(self, input, output):
    # input is the tuple of packed inputs
    # output is a Tensor
    print(f'Inside {self.__class__.__name__} forward')
    print('')
    print(f'Input Type: {type(input)}')
    print(f'Input[0] Type: {type(input[0])}')    
    print(f'Input[0] Size: {input[0].size()}')
    print(f'Output Type: {type(output)}')
    print(f'Output Size: {output.data.size()}')

In [38]:
model.conv2.register_forward_hook(printnorm)

In [39]:
out = model(inp)

Inside Conv2d forward

Input Type: <class 'tuple'>
Input[0] Type: <class 'torch.Tensor'>
Input[0] Size: torch.Size([1, 10, 12, 12])
Output Type: <class 'torch.Tensor'>
Output Size: torch.Size([1, 20, 8, 8])


## Module List

In [54]:
class LinearNet(nn.Module):    
    
    def __init__(self, input_size, num_layers, layers_size, output_size):
        super(LinearNet, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(input_size, layers_size)])
        self.linears.extend([nn.Linear(layers_size, layers_size) for i in range(1, num_layers-1)])
        self.linears.append(nn.Linear(layers_size, output_size))

In [60]:
m = LinearNet(3, 3, 100, 1); m

LinearNet(
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=1, bias=True)
  )
)

In [61]:
m.linears

ModuleList(
  (0): Linear(in_features=3, out_features=100, bias=True)
  (1): Linear(in_features=100, out_features=100, bias=True)
  (2): Linear(in_features=100, out_features=1, bias=True)
)

In [62]:
m.linears[0]

Linear(in_features=3, out_features=100, bias=True)